In [ ]:
'''
- natural language processing (NLP), text analytics, and Machine Learning
- sentiment analysis or opinion mining

                        *****  Problem Statement *****
To predict the sentiment for a number of movie reviews obtained from the Internet Movie Database (IMDb)

# 50,000 movie reviews with  “positive” and “negative” sentiment class labels based on the review content
# additional movie reviews that are unlabeled

Hence our task will be to predict the sentiment of 15,000 labeled movie reviews 
and use the remaining 35,000 reviews for training our supervised models. 

We will still predict sentiments for only 15,000 reviews in case of unsupervised models 
to maintain consistency and enable ease of comparison.
'''

In [ ]:
## Do please preprocess the document corpus
## (e.g. Cleaning text; Removing accented characters; 
##       Expanding contractions; Removing special characters;
##       Stemming and lemmatization; Removing stopwords)

In [ ]:
''' Unsupervised Lexicon-Based Models'''
# A lexicon model typically uses a lexicon, also known as a dictionary or vocabulary of words 
# specifically aligned toward sentiment analysis. 

In [1]:
import pandas as pd
import numpy as np
import text_normalizer as tn
import model_evaluation_utils as meu

np.set_printoptions(precision = 2, linewidth = 80)

In [10]:
dataset = pd.read_csv(r"movie_reviews.csv")

In [13]:
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
sample_review_ids = [7626, 3533, 13010]


In [15]:
# Reaaaaally time-consuming
norm_test_reviews = tn.normalize_corpus(test_reviews)

In [20]:
# with open('middle_result.txt', 'a') as the_file:
#     the_file.write(str(temp))

In [4]:
'''*******************************'''
''' Sentiment Analysis with AFINN '''
'''*******************************'''
from afinn import Afinn

In [5]:
afn = Afinn(emoticons = True)

In [21]:
# Build the model
for review, sentiment in zip(test_reviews[sample_review_ids], 
                             test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', afn.score(review))
    print('-' * 60)

REVIEW: no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative
Predicted Sentiment polarity: -7.0
------------------------------------------------------------
REVIEW: I don't care if some people voted this movie to be bad. If you want the Truth this is a Very Good Movie! It has every thing a movie should have. You really should Get this one.
Actual Sentiment: positive
Predicted Sentiment polarity: 3.0
------------------------------------------------------------
REVIEW: Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch the carrot
Actual Sentiment: positive
Predicted Sentiment polarity: -3.0
------------------------------------------------------------


In [ ]:
'''Predict sentiment for test dataset'''

In [22]:
sentiment_polarity = [afn.score(review) for review in test_reviews]
predicted_sentiments = ['positive' if score >= 1.0 else 'negative' \
                        for score in sentiment_polarity]


In [ ]:
'''Evaluate model performance'''

In [23]:
meu.display_model_performance_metrics(true_labels=test_sentiments,
                                      predicted_labels=predicted_sentiments,
                                      classes=['positive', 'negative'])


Model Performance metrics:
------------------------------
Accuracy: 0.7118
Precision: 0.7289
Recall: 0.7118
F1 Score: 0.7062

Model Classification report:
------------------------------
             precision    recall  f1-score   support

   positive       0.67      0.85      0.75      7510
   negative       0.79      0.57      0.67      7490

avg / total       0.73      0.71      0.71     15000


Prediction Confusion Matrix:
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       6376     1134
        negative       3189     4301


In [24]:
'''**************************************'''
''' Sentiment Analysis with SentiWordNet '''
'''**************************************'''
from nltk.corpus import sentiwordnet as swn

In [25]:
# toy example
awesome = list(swn.senti_synsets('awesome', 'a'))[0]
print('Positive Polarity Score:', awesome.pos_score())
print('Negative Polarity Score:', awesome.neg_score())
print('Objective Score:', awesome.obj_score())

Positive Polarity Score: 0.875
Negative Polarity Score: 0.125
Objective Score: 0.0


In [ ]:
''' Build model'''

# A generic function to extract and aggregate sentiment score
#  1. take in a movie review, 
#  2. tag each word with its corresponding POS tag, 
#  3. extract out sentiment scores for any matched synset token based on its POS tag, 
#  4. finally aggregates the scores.

In [38]:
def analyze_sentiment_swn_lexicon(review, verbose = False):
    
    # # tokenize and POS tag text tokens
    tagged_text = [(token.text, token.tag_) for token in tn.nlp(review)]
    pos_score = neg_score = token_count = obj_score = 0
    
    for word, tag in tagged_text:
        # # get wordnet synsets based on POS tags
        ss_set = None
        
        if 'NN' in tag and list(swn.senti_synsets(word, 'n')):
            ss_set = list(swn.senti_synsets(word, 'n'))[0]
            
        elif 'VB' in tag and list(swn.senti_synsets(word, 'v')):
            ss_set = list(swn.senti_synsets(word, 'v'))[0]
        
        elif 'JJ' in tag and list(swn.senti_synsets(word, 'a')):
            ss_set = list(swn.senti_synsets(word, 'a'))[0]
            
        elif 'RB' in tag and list(swn.senti_synsets(word, 'r')):
            ss_set = list(swn.senti_synsets(word, 'r'))[0]
        
        # # if senti-synset is found    
        if ss_set:
            # # add scores for all found synsets
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
        
        # # aggregate final scores
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 'positive' if norm_final_score >= 0 else 'negative'
        
    if verbose:
            norm_obj_score = round(float(obj_score) / token_count, 2)
            norm_pos_score = round(float(pos_score) / token_count, 2)
            norm_neg_score = round(float(neg_score) / token_count, 2)
            # display in a nice table
            sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score, 
                                             norm_pos_score, norm_neg_score, norm_final_score]],
                                            columns = pd.MultiIndex(levels = [['SENTIMENT STATS:'],
                                                                              ['Predicted Sentiment',
                                                                               'Objectivity',
                                                                               'Positive', 
                                                                               'Negative',
                                                                               'Overall']],
                                                                    labels = [[0,0,0,0,0],
                                                                              [0,1,2,3,4]]))
    print(sentiment_frame)
            
    return final_sentiment
    

In [ ]:
'''Predict sentiment for sample reviews'''

In [39]:
for review, sentiment in zip(test_reviews[sample_review_ids], 
                             test_sentiments[sample_review_ids]):
    
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_swn_lexicon(review, verbose=True)
    print('-'*60)
    

REVIEW: no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            negative        0.76     0.09     0.15   -0.06
------------------------------------------------------------
REVIEW: I don't care if some people voted this movie to be bad. If you want the Truth this is a Very Good Movie! It has every thing a movie should have. You really should Get this one.
Actual Sentiment: positive
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.76     0.19     0.06    0.13
------------------------------------------------------------
REVIEW: Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch 

In [ ]:
'''Predict sentiment for test dataset'''

In [40]:
predicted_sentiments = [analyze_sentiment_swd_lexicon(review, verbose=False) \
                        for review in norm_test_reviews]

In [ ]:
'''Evaluate model performance'''

In [43]:
meu.display_model_performance_metrics(true_labels=test_sentiments, 
                                      predicted_labels=predicted_sentiments, 
                                      classes=['positive', 'negative'])

Model Performance metrics:
------------------------------
Accuracy: 0.6851
Precision: 0.6905
Recall: 0.6851
F1 Score: 0.6827

Model Classification report:
------------------------------
             precision    recall  f1-score   support

   positive       0.66      0.77      0.71      7510
   negative       0.72      0.60      0.66      7490

avg / total       0.69      0.69      0.68     15000


Prediction Confusion Matrix:
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       5784     1726
        negative       2998     4492


In [42]:
'''*******************************'''
''' Sentiment Analysis with VADER '''
'''*******************************'''
# rule-based sentiment analysis
# specifically tuned to analyze sentiments in social media

# There were a total of over 9000 lexical features 
# over 7500 curated lexical features were selected in the lexicon with proper validated valence scores. 
# Each feature was rated on a scale from 
# {max}      "[-4] Extremely Negative"  
# {min}      "[4] Extremely Positive"
#           "[0] Neutral (or Neither, N/A)"

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [44]:
def analyze_sentiment_vader_lexicon(review,
                                    threshold=0.1,
                                    verbose=False):
    
    # pre-process text
    # (but keep the punctuations and emoticons intact. )
    review = tn.strip_html_tags(review)
    review = tn.remove_accented_chars(review)
    review = tn.expand_contractions(review)
    
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold else 'negative'
    
    if verbose:
        positive = str(round(scores['pos'], 2) * 100) + '%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns = pd.MultiIndex(levels = [['SENTIMENT STATS:'],
                                                                          ['Predicted Sentiment',
                                                                           'Polarity Score',
                                                                           'Positive', 
                                                                           'Negative',
                                                                           'Neutral']],
                                                                    labels = [[0,0,0,0,0],
                                                                              [0,1,2,3,4]]))
        
        print(sentiment_frame)

    return final_sentiment
    

In [ ]:
'''Predict sentiment for sample reviews'''

In [46]:
for review, sentiment in zip(test_reviews[sample_review_ids], 
                             test_sentiments[sample_review_ids]):
    
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=True)
    # We use a threshold of >= 0.4 for positive and < 0.4 for negative in our corpus.
    print('-'*60)

REVIEW: no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative
     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            negative           -0.8     0.0%    40.0%   60.0%
------------------------------------------------------------
REVIEW: I don't care if some people voted this movie to be bad. If you want the Truth this is a Very Good Movie! It has every thing a movie should have. You really should Get this one.
Actual Sentiment: positive
     SENTIMENT STATS:                                                     
  Predicted Sentiment Polarity Score Positive             Negative Neutral
0            negative          -0.16    16.0%  14.000000000000002%   69.0%
------------------------------------------------------------
REVIEW: Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unb

In [ ]:
'''Predict sentiment for test dataset'''

In [47]:
predicted_sentiments = [analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=False) \
                        for review in norm_test_reviews]

In [ ]:
'''Evaluate model performance'''

In [48]:
meu.display_model_performance_metrics(true_labels=test_sentiments, 
                                      predicted_labels=predicted_sentiments, 
                                      classes=['positive', 'negative'])

Model Performance metrics:
------------------------------
Accuracy: 0.7033
Precision: 0.7127
Recall: 0.7033
F1 Score: 0.6999

Model Classification report:
------------------------------
             precision    recall  f1-score   support

   positive       0.67      0.81      0.73      7510
   negative       0.76      0.60      0.67      7490

avg / total       0.71      0.70      0.70     15000


Prediction Confusion Matrix:
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       6079     1431
        negative       3020     4470
